# Imports

In [12]:
import os
import pandas as pd


from dotenv import load_dotenv
import requests
from sqlalchemy import create_engine
from bs4 import BeautifulSoup

from packages.utils import extract_player_salaries, load_into_db, grab_all_salaries

In [13]:

dotenv_path = ("/Users/maukanmir/Documents/Machine-Learning/Web-Scraping-Code/Player-Salaries/dot.env")
load_dotenv(dotenv_path)

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
TABLE_NAME_PlAYER_SALARIES = "player_salaries"
TABLE_NAME_MASTER_PLAYER_SALARIES = "master_player_salaries"

engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}')

# Grabbing Player Salaries from 2011-2024

In [20]:
years = [str(year) for year in range(2011, 2025)]
all_data = pd.DataFrame()

for year in years:
  url = f"https://www.spotrac.com/nba/rankings/player/_/year/{year}/sort/cash_total"
  response = requests.get(url)
  year = int(year)
  df = extract_player_salaries(response.text, year)
  all_data = pd.concat([all_data, df])

# Download Dataset into DB

In [21]:
load_into_db(all_data, engine, TABLE_NAME_PlAYER_SALARIES)

Data successfully written to the database.


# Grabbing Player Salaries From 1989-2024

In [3]:
all_data = pd.DataFrame()
years = [str(year) + "-" + str(year+1) for year in range(1989, 2025)]
for year in years:
  base_url = "https://hoopshype.com/salaries/players/{year}/"
  response = requests.get(base_url)
  df = grab_all_salaries(response.text, year)
  all_data = pd.concat([all_data, df])


In [4]:
load_into_db(all_data, engine, TABLE_NAME_MASTER_PLAYER_SALARIES)

Data successfully written to the database.


In [14]:
all_data = pd.DataFrame()
for year in years:
  base_url = "https://hoopshype.com/salaries/players/{year}/"
  response = requests.get(base_url)
  df = grab_all_salaries(response.text, year)
  all_data = pd.concat([all_data, df])

In [8]:
all_data

,name,salary,season
0,Stephen Curry,55761217,1989-1990
1,Kawhi Leonard,52368085,1989-1990
2,Nikola Jokic,51415938,1989-1990
3,Joel Embiid,51415938,1989-1990
4,Kevin Durant,51179020,1989-1990
...,...,...,...
494,Tristan Vukcevic,578577,1990-1991
495,Juwan Morgan,576230,1990-1991
496,Malik Fitts,555217,1990-1991
497,Didi Louzada,268032,1990-1991
